In [4]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.offline as py
import numpy as np
import pandas as pd

# Read data from file
gps_data = "/mnt/hdd02/ba_copter_videos/Flight_05/DJIFlightRecord_2024-04-12_[18-34-58].csv"
# File name of trajectory data
slam_data = '/home/bastian/uni/visual_odometry_playground/orb_slam3/orb_slam3_ws/CameraTrajectory.csv'

######################## GPS PLOT ########################

data = pd.read_csv(gps_data)
# print(data.columns)

# Extracting data
latitude = data["OSD.latitude"]
longitude = data["OSD.longitude"]
height = data["OSD.height [ft]"]

# Create subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}]], subplot_titles=['GPS','SLAM'])

gps_layout = dict(title='GPS',
                  scene=dict(xaxis=dict(title='Longitude'),  # Ändere die Beschriftung der Achsen
                             yaxis=dict(title='Latitude'),
                             zaxis=dict(title='Height [ft]')),  # Ändere die Beschriftung der Achsen
                  margin=dict(l=0, r=0, t=30, b=0))

# Adjust layout
fig.update_scenes(xaxis=dict(title='Longitude'),
                  yaxis=dict(title='Latitude'),
                  zaxis=dict(title='Height [ft]'),
                  row=1, col=1)

# Add trace for trajectory
fig.add_trace(go.Scatter3d(
    x=longitude,
    y=latitude,
    z=height,
    mode='lines', name='GPS Tracking'
), row=1, col=1)

######################## SLAM PLOT ########################

# Lists for delta values
cam_delta_twb_x = []
cam_delta_twb_y = []
cam_delta_twb_z = []

# Read trajectory data and store delta values in lists
with open(slam_data, 'r') as f:
    for line in f:
        parts = line.split()
        cam_delta_twb_x.append(float(parts[1]))
        cam_delta_twb_y.append(float(parts[2]))
        cam_delta_twb_z.append(float(parts[3]))

# Rotation matrix for x-axis
theta_x = np.radians(0)
rotation_matrix_x = np.array([[1, 0, 0],
                            [0, np.cos(theta_x), -np.sin(theta_x)],
                            [0, np.sin(theta_x), np.cos(theta_x)]])

# Rotation matrix for the y-axis
theta_y = np.radians(180)
rotation_matrix_y = np.array([[np.cos(theta_y), 0, np.sin(theta_y)],
                            [0, 1, 0],
                            [-np.sin(theta_y), 0, np.cos(theta_y)]])

# Rotation matrix for the z-axis
theta_z = np.radians(0)
rotation_matrix_z = np.array([[np.cos(theta_z), -np.sin(theta_z), 0],
                            [np.sin(theta_z), np.cos(theta_z), 0],
                            [0, 0, 1]])

# Rotate data around the x-axis
rot_cam_coords_x = np.dot(rotation_matrix_x, np.vstack([cam_delta_twb_x, cam_delta_twb_y, cam_delta_twb_z]))

# Rotate data around the y-axis
rot_cam_coords_y = np.dot(rotation_matrix_y, rot_cam_coords_x)

# Rotate data around the z-axis
rot_cam_coords = np.dot(rotation_matrix_z, rot_cam_coords_y)

slam_layout = dict(title='SLAM',
                  scene=dict(xaxis=dict(title='X'), 
                             yaxis=dict(title='Y'),
                             zaxis=dict(title='Z')), 
                  margin=dict(l=0, r=0, t=30, b=0))

# Adjust layout
fig.update_scenes(xaxis=dict(title='X'),
                  yaxis=dict(title='Y'),
                  zaxis=dict(title='Z'),
                  row=1, col=2)

# Add plot
fig.add_trace(go.Scatter3d(
    x=rot_cam_coords[0], 
    y=rot_cam_coords[1], 
    z=rot_cam_coords[2], 
    #x=cam_delta_twb_x,
    #y=cam_delta_twb_y,
    #z=cam_delta_twb_z,
    mode='lines', name='SLAM Tracking'
), row=1, col=2)

fig.update_layout(title='GPS vs SLAM',
                  width=750,
                  height=400,
                  margin=dict(l=0, r=0, t=60, b=0))

py.iplot(fig)